# Testing the stability of a CKSVAR model

Here, we test the stability of the CKSVAR model estimated for Japan in:
- Ikeda, Li, Mavroeidis and Zanetti (2021), "[Testing the effectiveness of monetary policy in Japan and the United States](https://arxiv.org/abs/2012.15158)." Working paper.

This paper is hereafter referred to as ILMZ21.

In ILMZ21, the effectiveness of unconventional monetary policy in Japan is investigated via estimation of a CKSVAR(2) model in inflation, output gap and the Call Rate for Japan, over the period 1985q3-2019q1, with shadow call rate as the latent variable.

Converting this model into TAR form, we determine upper bounds on the state-constrained joint spectral radius (SCJSR)

In [1]:
using DataFrames, DataFramesMeta, CSV, HTTP
using ThresholdStability

extract_est(parname, df) = @subset(df, in([parname]).(:Parameter)).:Estimate[1]

JP = CSV.read(HTTP.get("https://raw.githubusercontent.com/samwycherley/ThresholdStability.jl/master/examples/src/estimates/estimates_JP.csv").body, DataFrame, copycols=true)  # Japan data from CSV

┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1342
┌ Info: Precompiling DataFramesMeta [1313f7d8-7da2-5740-9ea0-a2ca25f37964]
└ @ Base loading.jl:1342
┌ Info: Precompiling ThresholdStability [da96a6fb-8b33-4fd1-b1ac-41ea4d13f1f8]
└ @ Base loading.jl:1342


,Parameter,Estimate
,String15,Float64
1,tau,0.357388
2,Eq.3 Constant,-0.125709
3,Eq.3 PGR,0.132664
4,Eq.3 INFL_1,0.295522
5,Eq.3 INFL_2,0.0888194
6,Eq.3 YGAP_BOJ_1,0.175247
7,Eq.3 YGAP_BOJ_2,-0.145538
8,Eq.3 SR_1,1.23818
9,Eq.3 SR_2,-0.358167


This loads the Japan estimates as a dataframe and defines a function to extract estimates. Next, we build the matrices:

In [2]:
JPβtilde = zeros(2)
for i in 1:2
    JPβtilde[i] = extract_est("tildebeta_$i", JP)
end

JPCstar = zeros(3, 2)
for i in 1:3
    for j in 1:2
        JPCstar[i, j] = extract_est("Eq.$i lSR_$j", JP)
    end
end

Cbars = []
for j in 1:2
    Cbar_j = zeros(3, 3)
    for i in 1:3
        Cbar_j[i, 1] = extract_est("Eq.$i INFL_$j", JP)
        Cbar_j[i, 2] = extract_est("Eq.$i YGAP_BOJ_$j", JP)
        Cbar_j[i, 3] = extract_est("Eq.$i SR_$j", JP)
    end
    push!(Cbars, Cbar_j)
end
JPCbar = hcat(Cbars[1], Cbars[2])

JPC = copy(JPCbar); k = size(JPC, 1)  # convert C̄ to C
for i in 1:2
    JPC[:, i*k] -= JPCstar[:, i]
end

Σ_JP, X_JP = CKSVAR_to_TAR(JPC, JPCstar, JPβtilde, 2)

([[-0.00075365356865 0.161144327919405 … -0.063244772053654 -0.0019747468909369903; 0.091260254002804 1.51572351460357 … -0.593269170973906 -0.19966646656439402; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.00075365356865 0.161144327919405 … -0.063244772053654 0.210848444745381; 0.091260254002804 1.51572351460357 … -0.593269170973906 0.786926449180816; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.00075365356865 0.161144327919405 … -0.063244772053654 -0.0019747468909369903; 0.091260254002804 1.51572351460357 … -0.593269170973906 -0.19966646656439402; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.00075365356865 0.161144327919405 … -0.063244772053654 0.210848444745381; 0.091260254002804 1.51572351460357 … -0.593269170973906 0.786926449180816; … ; 0.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], Any[[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0], [0.0 0.0 … 0.0 0.0]], [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 -1.0], [0.0 0.0 … 0.0 0.0]], [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0], [0.0 0.0 … 0.0 0.0]], [[0.0 0

Write this as a state-dependent switched linear system:

In [3]:
G_JP = automaton_constructor(Σ_JP)
s_JP = discreteswitchedsystem(Σ_JP, G_JP, X_JP)

Hybrid System with automaton LightAutomaton{LightGraphs.SimpleGraphs.SimpleDiGraph{Int64}, LightGraphs.SimpleGraphs.SimpleEdge{Int64}}({4, 8} directed simple Int64 graph, Dict{LightGraphs.SimpleGraphs.SimpleEdge{Int64}, Dict{Int64, Int64}}(Edge 3 => 2 => Dict(5 => 3), Edge 1 => 1 => Dict(1 => 1), Edge 4 => 2 => Dict(7 => 4), Edge 1 => 3 => Dict(2 => 1), Edge 2 => 1 => Dict(3 => 2), Edge 3 => 4 => Dict(6 => 3), Edge 2 => 3 => Dict(4 => 2), Edge 4 => 4 => Dict(8 => 4)), 8, 8)

In [4]:
@show sosbound_γ(s_JP, 2)

┌ Warning: Making slow progress. Reporting upper bound with reduced accuracy.
└ @ ThresholdStability C:\Users\samvp\.julia\packages\ThresholdStability\3zSAB\src\sos_gamma_search.jl:43


sosbound_γ(s_JP, 2) = 0.9409652709960938


0.9409652709960938

We see that the model estimated for Japan is stable